## MFE 230I Problem Set 3

### Tree-Based Models

### Q1


In [1]:

from __future__ import print_function, division

import pandas as pd
import numpy as np

import statsmodels.api as sm
from statsmodels import tsa
from datetime import date, datetime, timedelta
import copy
import scipy as sp
from scipy.optimize import fsolve

from cycler import cycler
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl
import matplotlib.dates

# Plotting and display options
np.set_printoptions(precision=10)
pd.set_option('precision', 8)
pd.set_option('display.float_format', lambda x: '%.8f' % x)

plt.style.use('ggplot')

mpl.rcParams['lines.linewidth'] = 1.5
mpl.rcParams['lines.color'] = 'blue'
mpl.rcParams['axes.prop_cycle'] = cycler('color', ['#30a2da', '#e5ae38', '#fc4f30', '#6d904f', '#8b8b8b'])
mpl.rcParams['legend.fancybox'] = True
mpl.rcParams['legend.fontsize'] = 14
mpl.rcParams['axes.facecolor'] = '#f0f0f0'
mpl.rcParams['axes.labelsize'] = 15
mpl.rcParams['axes.axisbelow'] = True
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['axes.labelpad'] = 0.0
mpl.rcParams['axes.xmargin'] = 0.05  # x margin.  See `axes.Axes.margins`
mpl.rcParams['axes.ymargin'] = 0.05  # y margin See `axes.Axes.margins`
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['figure.subplot.left'] = 0.08
mpl.rcParams['figure.subplot.right'] = 0.95
mpl.rcParams['figure.subplot.bottom'] = 0.07

# figure configuration
fsize = (10,7.5) # figure size
tsize = 18 # title font size
lsize = 16 # legend font size
csize = 14 # comment font size
grid = True # grid

# this allows plots to appear directly in the notebook
get_ipython().magic(u'matplotlib inline')



In [2]:
#initial values

maturity = [1,2,3,4,5,6]
yields = [0.05, 0.055, 0.0570, 0.0590 , 0.060 , 0.0610]
volatility = 0.015

#find next drift for Binomial Tree

def find(m, BT, y, volatility):
    values=[]
    T=len(BT)+1
    Bt=BT[:]
    Bt.reverse()
    for i in Bt:
        temp=[]
        if len(values)==0:
            for r in i:
                values.append((0.5*100/(1+r+volatility+m)+0.5*100/(1+r+m-volatility))/(1+r))
        else:
            for ind in range(len(i)):
                temp.append((values[ind]+values[ind+1])/(2*(1+i[ind])))
            values=temp[:]
        
    return(values[0] - 100/((1+y)**T))

def generate_BT(maturity = maturity, yields = yields, volatility = volatility):
    BT=[[yields[0]]]
    for matur, y in zip(maturity[1:],yields[1:]):
        m = fsolve(find, 0, args=(BT, y , volatility))[0]
        last = BT[-1]
        last_last = last[-1]
        new_branch = [knots + m + volatility for knots in last]
        new_branch.append(last_last - volatility + m)
        BT.append(new_branch)
        
    return BT



In [4]:
BT=generate_BT()
#for br in BT:
 #   print(format(br))

pd.DataFrame(BT).T


,0,1,2,3,4,5
0,0.05000000,0.07523603,0.09164759,0.11129248,0.12612463,0.14418405
1,nan,0.04523603,0.06164759,0.08129248,0.09612463,0.11418405
2,nan,nan,0.03164759,0.05129248,0.06612463,0.08418405
3,nan,nan,nan,0.02129248,0.03612463,0.05418405
4,nan,nan,nan,nan,0.00612463,0.02418405
5,nan,nan,nan,nan,nan,-0.00581595


## Q2

### (a)

The mortgagae value is **98.90748**

In [10]:
#coupon every year is:
c=100/sum([1/(1.055**i) for i in range(1,7)])
c

20.01789476223801

In [11]:
Bt=BT[:]
Bt.reverse()
values=[]
value_tree=[]
for branch in Bt:
    temp=[]
    if len(values)==0:
        for r in branch:
            values.append(c/(1+r))
    else:
        for ind in range(len(branch)):
            temp.append((0.5*values[ind]+0.5*values[ind+1]+c)/(1+branch[ind]))
        values=temp[:]
    value_tree.append(values)
value_tree.reverse()
print(values)

[98.907483369778262]


In [17]:
print("Value by Tree")
pd.DataFrame(value_tree).T

Value by Tree


,0,1,2,3,4,5
0,98.01705337,80.33125436,64.24577241,48.78859447,33.52095716,17.49534509
1,nan,85.46876820,68.46855552,51.44310187,34.88004990,17.96641659
2,nan,nan,70.16572626,53.90006319,36.33423928,18.46355774
3,nan,nan,nan,54.00694645,36.95943374,18.97430783
4,nan,nan,nan,nan,36.95943374,18.97430783
5,nan,nan,nan,nan,nan,18.97430783


In [13]:
#the spot rate duration:
print("Spot rate duration is: ")
print(-(value_tree[1][0]-value_tree[1][1])/(BT[1][0]-BT[1][1])/values[0])

Spot rate duration is: 
2.29308540755


### (b) 

In [14]:
#backward from value tree
#value_tree[-1]

In [15]:
#calculate the mortgage value
table=[]
mortgage_value=[100,c,5.5,c-5.5]
table.append(mortgage_value)
for i in range(2,7):
    new_value=[]
    new_value.append(mortgage_value[0]-mortgage_value[3])
    new_value.append(c)
    new_value.append(new_value[0]*0.055)
    new_value.append(new_value[1]-new_value[2])
    table.append(new_value)
    mortgage_value=new_value

Mortgage_=pd.DataFrame(table,columns=['Principal not due','repayment','interest part','principal part'],index=[1,2,3,4,5,6])
Mortgage_

,Principal not due,repayment,interest part,principal part
1,100.00000000,20.01789476,5.50000000,14.51789476
2,85.48210524,20.01789476,4.70151579,15.31637897
3,70.16572626,20.01789476,3.85911494,16.15877982
4,54.00694645,20.01789476,2.97038205,17.04751271
5,36.95943374,20.01789476,2.03276886,17.98512591
6,18.97430783,20.01789476,1.04358693,18.97430783


We assume that the borrower refinance optimally, so this means for example, in the last payment, if the value of mortgage is larger than 18.974308, the homeowener will prepay their mortgage. It acts like a callable bonds. Lender will no longer benefits from lower interests protection.

In [19]:
Bt=BT[:]
Bt.reverse()
values=[]
value_tree=[]
prepaid_tree=[]
optimal_value=Mortgage_['Principal not due'].tolist()
optimal_value.reverse()
k=0
for branch in Bt:
    opt=optimal_value[k]
    k=k+1
    temp=[]
    if len(values)==0:
        for r in branch:
            values.append(min(c/(1+r),opt))
    else:
        for ind in range(len(branch)):
            temp.append(min((0.5*values[ind]+0.5*values[ind+1]+c)/(1+branch[ind]),opt))
        values=temp[:]
    value_tree.append(values)
    prepaid_tree.append(np.equal(values,opt).tolist())
value_tree.reverse()
prepaid_tree.reverse()
print(pd.DataFrame(value_tree).T)
print("Mortgage value:", values[0])

            0           1           2           3           4           5
0 98.01705337 80.33125436 64.24577241 48.78859447 33.52095716 17.49534509
1         nan 85.46876820 68.46855552 51.44310187 34.88004990 17.96641659
2         nan         nan 70.16572626 53.90006319 36.33423928 18.46355774
3         nan         nan         nan 54.00694645 36.95943374 18.97430783
4         nan         nan         nan         nan 36.95943374 18.97430783
5         nan         nan         nan         nan         nan 18.97430783
Mortgage value: 98.0170533713


In [20]:
#print(prepaid_tree)

In [21]:
#the spot rate duration:
print("Spot rate duration is: ")
print(-(value_tree[1][0]-value_tree[1][1])/(BT[1][0]-BT[1][1])/values[0])

Spot rate duration is: 
1.7471496601


### (c)


In [25]:
Bt=BT[:]
Pt=prepaid_tree[:]
Pt.reverse()
Bt.reverse()
values=[]
value_tree=[]
optimal_value=Mortgage_['Principal not due'].tolist()
principal_value=Mortgage_['principal part'].tolist()
principal_value.reverse()
optimal_value.reverse()
k=0
for branch,mask in zip(Bt,Pt):
    opt=optimal_value[k]
    prin=principal_value[k]
    k=k+1
    temp=[]
    if len(values)==0:
        for ind in range(len(branch)):
            if mask[ind]:
                values.append(opt)
            else:
                
                values.append(opt/(1+branch[ind]))
    else:
        for ind in range(len(branch)):
            if mask[ind]:
                temp.append(opt)
            else:
                
                temp.append(min((0.5*values[ind]+0.5*values[ind+1]+prin)/(1+branch[ind]),opt))
        values=temp[:]
    value_tree.append(values)

value_tree.reverse()
print("PO part:", values[0])

PO part: 83.1940813579


In [26]:
#the spot rate duration:
print("Spot rate duration is: ")
print(-(value_tree[1][0]-value_tree[1][1])/(BT[1][0]-BT[1][1])/values[0])

Spot rate duration is: 
3.4928422803


### (d)

In [28]:
Bt=BT[:]
Pt=prepaid_tree[:]
Pt.reverse()
Bt.reverse()
values=[]
value_tree=[]
optimal_value=Mortgage_['Principal not due'].tolist()
interest_value=Mortgage_['interest part'].tolist()
interest_value.reverse()
optimal_value.reverse()
k=0
for branch,mask in zip(Bt,Pt):
    opt=optimal_value[k]
    
    interest=interest_value[k]
    k=k+1
    temp=[]
    if len(values)==0:
        for ind in range(len(branch)):
            if mask[ind]:
                values.append(0)
            else:
                    
                values.append(interest/(1+branch[ind]))
    else:
        for ind in range(len(branch)):
            if mask[ind]:
                temp.append(0)
            else:
                
                temp.append((0.5*values[ind]+0.5*values[ind+1]+interest)/(1+branch[ind]))
        values=temp[:]
    
    value_tree.append(values)

value_tree.reverse()
print("IO part:", values[0])

IO part: 14.8229720134


To test the result is good just add PO part and IO part together, then it should be the mortgage value.

In [29]:
14.822972013373075+83.194081357893893

98.01705337126697

In [30]:
#the spot rate duration:
print("Spot rate duration is: ")
print(-(value_tree[1][0]-value_tree[1][1])/(BT[1][0]-BT[1][1])/values[0])


Spot rate duration is: 
-8.05056794604


| Summary |Price | Spot Rate Duration |
| ------| ------ | :------: |
| (a) | 98.90748 | 2.29308 |
| (b) | 98.01705 | 1.74714 |
| (c) | 83.19408 | 3.49284 |
| (d) | 98.01705 | -8.05056 |

## Q3

### (a) Calibration

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import seaborn as sns
from scipy.special import comb

def OLS(y, X, const=True):
    X = np.c_[np.ones(len(y)),X] if const else X
    var = X.T.dot(X)
    cov = X.T.dot(y)
    return np.linalg.inv(var).dot(cov)

def plot(x, y, title):
    plt.xlabel('time')
    plt.ylabel('rate')
    plt.title(title)
    plt.plot(x,y)
    plt.show()
    
data = pd.read_excel('HW1_data.xls',skiprows=0,convert_float=True)
data['T2'] = data['Maturity']**2
data['T3'] = data['Maturity']**3
data['T4'] = data['Maturity']**4
data['T5'] = data['Maturity']**5
data['log_Z'] = np.log(data['Price'] / 100)
data.head()
#a
X = data[['Maturity', 'T2', 'T3', 'T4', 'T5']].copy()
betas = OLS(data['log_Z'], X, const=False)
print(betas)
#b
T = np.arange(0.5, 10.1, 0.5)
Time = pd.DataFrame(T, columns=['T'])
Time['T2'] = Time['T']**2
Time['T3'] = Time['T']**3
Time['T4'] = Time['T']**4
Time['T5'] = Time['T']**5
Time.head()

Z = np.exp((betas*Time).sum(axis=1))
disc = Z


[ -3.2611556847e-02  -1.0778919536e-03  -1.9882987749e-05
   2.8532567382e-06  -4.7816045094e-08]


In [32]:
df=pd.read_excel('HW1_data.xls')
df.head()
df['discount']=df['Price']/100
df['T2']=np.power(df['Maturity'],2)
df['T3']=np.power(df['Maturity'],3)
df['T4']=np.power(df['Maturity'],4)
df['T5']=np.power(df['Maturity'],5)
df['logZ']=np.log(df['discount'])
olsm=sm.OLS(df['logZ'],df[['Maturity','T2','T3','T4','T5']])
res=olsm.fit()
res.summary()
def discou(T,pram=res.params):
    return np.exp(pram[0]*T+pram[1]*T**2+pram[2]*T**3+pram[3]*T**4+pram[4]*T**5)

disc=[discou((T+1)/2) for T in range(20)]

In [33]:
sigma=np.sqrt(2)*0.15/2
sigma

0.10606601717798213

In [34]:
r0=(1/disc[0]-1)*2


In [35]:
def discoutTree(BT):
    Bt = BT[:]
    Bt.reverse()
    values = []
    for branch in Bt:
        temp=[]
        if len(values)==0:
            for r in branch:
                values.append(1/(1+r/2))
        else:
            for ind in range(len(branch)):
                temp.append((0.5*values[ind]+ 0.5*values[ind+1])/(1+branch[ind]/2))
            values=temp[:]
    return(values[0])

def calculateVolatility(BT):
    T = len(BT)
    u_Bt = BT[:]
    upper_subtree = [i[:-1] for i in u_Bt[1:]]
    d_Bt = BT[:]
    down_subtree = [i[1:] for i in u_Bt[1:]]
    ZCB_u = discoutTree(upper_subtree)
    ZCB_d = discoutTree(down_subtree)

    r_u = 2*(1/ZCB_u)**(1/(T-1)) - 2
    r_d = 2*(1/ZCB_d)**(1/(T-1)) - 2
    
    return(np.log(r_u/r_d)/(2*np.sqrt(0.5)))


    
def find_BDT(m2, disc, Bt):
    BT=Bt[:]
    m, sigma = m2
    new_branch = [i*np.exp(m+sigma) for i in BT[-1]]
    last = BT[-1][-1]*np.exp(m-sigma)
    new_branch.append(last)
    BT.append(new_branch)
    Bt = BT[:]
    Bt.reverse()
    values = []
    for branch in Bt:
        temp=[]
        if len(values)==0:
            for r in branch:
                values.append(1/(1+r/2))
        else:
            for ind in range(len(branch)):
                temp.append((0.5*values[ind]+ 0.5*values[ind+1])/(1+branch[ind]/2))
            values=temp[:]
            
    vol = calculateVolatility(BT)
    return([values[0] - disc,vol - 0.15])
    
    
def generate_new_tree(Bt , m , sigma=sigma):
    BT=Bt[:]
    new_branch = [i*np.exp(m+sigma) for i in BT[-1]]
    last = BT[-1][-1]*np.exp(m-sigma)
    new_branch.append(last)
    BT.append(new_branch)
    return BT

In [37]:
bt = [[r0]]
mt= []
sig_t = []
for dis in disc[1:]:
    [m,sig] = sp.optimize.fsolve(find_BDT,[0.1, 0.1] ,xtol=1e-12,args=(dis ,bt))
    mt.append(m)
    sig_t.append(sig)
    BT = generate_new_tree(bt, m , sigma = sig)
    bt = BT[:]
    
pd.DataFrame(bt).T



BT= bt[:]
BT.reverse()
values = []
value_tree = []
for branch in BT:
    temp = []

    if len(values)==0:
        for r in branch:
            values.append(100/(1+r/2))
    else:
        for ind in range(len(branch)):
            temp.append((0.5*values[ind]+0.5*values[ind+1])/(1+branch[ind]/2))
        values = temp[:]
    value_tree.append(values)

    
value_tree.reverse()


(pd.DataFrame(bt).T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.03343146,0.03821148,0.04366112,0.04986403,0.05691362,0.06491422,0.07398241,0.08424853,0.09585833,0.10897497,0.12378112,0.14048154,0.15930581,0.18051167,0.20438860,0.23126209,0.26149845,0.29551026,0.33376296,0.37678110
1,nan,0.03090766,0.03531565,0.04033292,0.04603503,0.05250638,0.05984126,0.06814508,0.07753577,0.08814526,0.10012134,0.11362960,0.12885576,0.14600829,0.16532133,0.18705817,0.21151508,0.23902581,0.26996680,0.30476236
2,nan,nan,0.02856269,0.03262057,0.03723234,0.04246626,0.04839858,0.05511458,0.06270960,0.07129038,0.08097642,0.09190167,0.10421633,0.11808900,0.13370906,0.15128944,0.17106978,0.19331998,0.21834452,0.24648658
3,nan,nan,nan,0.02637470,0.03010345,0.03433523,0.03913169,0.04456177,0.05070257,0.05764039,0.06547184,0.07430523,0.08426200,0.09547846,0.10810774,0.12232200,0.13831499,0.15630494,0.17653803,0.19929171
4,nan,nan,nan,nan,0.02432586,0.02774545,0.03162135,0.03600926,0.04097150,0.04657777,0.05290618,0.06004422,0.06809004,0.07715378,0.08735919,0.09884539,0.11176892,0.12630615,0.14265601,0.16104270
5,nan,nan,nan,nan,nan,0.02240161,0.02553100,0.02907379,0.03308028,0.03760678,0.04271631,0.04847955,0.05497572,0.06229377,0.07053359,0.07980752,0.09024195,0.10197927,0.11518011,0.13002547
6,nan,nan,nan,nan,nan,nan,0.02059014,0.02344731,0.02667845,0.03032896,0.03444968,0.03909759,0.04433660,0.05023842,0.05688364,0.06436284,0.07277796,0.08224383,0.09288998,0.10486241
7,nan,nan,nan,nan,nan,nan,nan,0.01888194,0.02148396,0.02442368,0.02774207,0.03148499,0.03570393,0.04045662,0.04580797,0.05183091,0.05860754,0.06623033,0.07480360,0.08444491
8,nan,nan,nan,nan,nan,nan,nan,nan,0.01726949,0.01963253,0.02229995,0.02530864,0.02869995,0.03252032,0.03682190,0.04166333,0.04711060,0.05323805,0.06012951,0.06787950
9,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.01574689,0.01788638,0.02029959,0.02301970,0.02608394,0.02953416,0.03341738,0.03778654,0.04270124,0.04822876,0.05444488


In [41]:
#print(pd.DataFrame([i[:-1] for i in bt[1:]]).T)
#test if the volatility is the constant at 0.15
for i in range(1,11):
    print(i) 
    print(calculateVolatility(bt[:10]))

1
0.15
2
0.15
3
0.15
4
0.15
5
0.15
6
0.15
7
0.15
8
0.15
9
0.15
10
0.15


### (b)

We assume the bond pay semi-annual coupon

In [42]:
BT= bt[:]
BT.reverse()
values = []
value_tree = []
count = 17 # dirty trick on call protection
for branch in BT:
    temp = []
    count = count -1
    if len(values)==0:
        for r in branch:
            values.append(min(100, 102/(1+r/2)))
    else:
        for ind in range(len(branch)):
            if count > 0:
                temp.append(min((0.5*values[ind]+0.5*values[ind+1]+2)/(1+branch[ind]/2),100))
            else:
                temp.append( (0.5*values[ind]+0.5*values[ind+1]+2)/(1+branch[ind]/2))
        values = temp[:]
    value_tree.append(values)

    
value_tree.reverse()
callable_value_tree = value_tree
print(values[0])
pd.DataFrame(callable_value_tree).T

96.0691359369


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,96.06913594,93.01116133,89.69677984,86.25574527,82.81395729,79.46042390,76.26099375,73.26832155,70.52944677,68.09191136,66.00899212,64.34465118,63.17918154,62.61699250,62.79839953,63.91797814,66.25371144,70.21436637,76.41853603,85.83036961
1,nan,98.33884205,95.87963726,93.05407668,89.99859262,86.88073271,83.81796526,80.89563805,78.17994439,75.72781737,73.59514452,71.84400033,70.54935620,69.80618133,69.73868306,70.51409767,72.36405031,75.61829926,80.75926256,88.51237906
2,nan,nan,99.83747032,98.09124908,95.86270311,93.25954045,90.50529267,87.75606481,85.12397183,82.69381355,80.53553866,78.71473316,77.30224938,76.38322218,76.06596044,76.49256026,77.85438312,80.41588957,84.55206109,90.80846597
3,nan,nan,nan,100.43531815,99.51958774,98.03505888,95.97417194,93.63484844,91.22479619,88.89222049,86.74736176,84.87782408,83.36123257,82.27743357,81.72050221,81.81006861,82.70325425,84.61140857,87.82571611,92.75713581
4,nan,nan,nan,nan,100.00000000,100.00000000,99.46200207,98.06912701,96.21743535,94.18270384,92.16086118,90.29640896,88.70128174,87.46921555,86.69008756,86.46555472,86.92405447,88.23419969,90.62228205,94.39887501
5,nan,nan,nan,nan,nan,100.00000000,100.00000000,100.00000000,99.45221584,98.19433909,96.59136688,94.90119210,93.31331341,91.97302138,90.99692430,90.48779638,90.55377606,91.32931712,92.99063979,95.77350249
6,nan,nan,nan,nan,nan,nan,100.00000000,100.00000000,100.00000000,100.00000000,99.49008372,98.40756873,97.08983791,95.78357240,94.67846465,93.92439215,93.64342294,93.94998403,94.98169169,96.91844885
7,nan,nan,nan,nan,nan,nan,nan,100.00000000,100.00000000,100.00000000,100.00000000,100.00000000,99.57279826,98.70073643,97.70069589,96.81819310,96.25060186,96.15203875,96.64508250,97.86778203
8,nan,nan,nan,nan,nan,nan,nan,nan,100.00000000,100.00000000,100.00000000,100.00000000,100.00000000,100.00000000,99.69387555,99.05866904,98.40395923,97.99017585,98.02717647,98.65178309
9,nan,nan,nan,nan,nan,nan,nan,nan,nan,100.00000000,100.00000000,100.00000000,100.00000000,100.00000000,100.00000000,100.00000000,99.84049298,99.45361236,99.16998068,99.29689603


### (c) Forward price

In [44]:
p=disc[19] / disc[1]
p

0.67278446386163215

### (d) Future price

In [45]:
BT= bt[:]
BT.reverse()
values = []
value_tree = []
for branch in BT:
    temp = []

    if len(values)==0:
        for r in branch:
            values.append(100/(1+r/2))
    else:
        for ind in range(len(branch)):
            temp.append((0.5*values[ind]+0.5*values[ind+1])/(1+branch[ind]/2))
        values = temp[:]
    value_tree.append(values)

    
value_tree.reverse()

ZCB_value_tree = value_tree
print(values[0])

65.0485163778


In [46]:
disc[19]

0.65048516377782761

In [47]:
#future price is the q-measure of value_tree[2] without discount

0.25*value_tree[2][0]+0.5*value_tree[2][1]+0.25* value_tree[2][2]

67.273434984218284

The future is mark to market, thus we dont need to discount it, while the forward need to discount.

### (e) Hedging with forward

In [48]:
#caculate the delta of callable bond using the binomial tree

D_call = (callable_value_tree[1][0] - callable_value_tree[1][1])/(bt[1][0] - bt[1][1])
D_call

-729.43722863414575

In [49]:
#A forward contract is like long a 10-year zcb and short some amount of a 1-year zcb
#First caculate the delta of a 10-year ZCB

BT= bt[:]
BT.reverse()
values = []
value_tree = []
for branch in BT:
    temp = []

    if len(values)==0:
        for r in branch:
            values.append(100/(1+r/2))
    else:
        for ind in range(len(branch)):
            temp.append((0.5*values[ind]+0.5*values[ind+1])/(1+branch[ind]/2))
        values = temp[:]
    value_tree.append(values)

    
value_tree.reverse()

D_10 = ( value_tree[1][0] -  value_tree[1][1])/(bt[1][0] - bt[1][1])
D_10

-784.04032867014871

In [50]:
#next build a 1-year zcb value tree

BT= bt[:2]
BT.reverse()
values = []
value_tree = []
for branch in BT:
    temp = []

    if len(values)==0:
        for r in branch:
            values.append(100/(1+r/2))
    else:
        for ind in range(len(branch)):
            temp.append((0.5*values[ind]+0.5*values[ind+1])/(1+branch[ind]/2))
        values = temp[:]
    value_tree.append(values)

    
value_tree.reverse()


print(values[0])

96.6855209534


In [51]:
#A forward contract is like long a 10-year zcb and short some amount of a 1-year zcb
#Then, caculate the delta of a 1-year ZCB

D_1 = ( value_tree[1][0] -  value_tree[1][1])/(bt[1][0] - bt[1][1])
D_1

-48.315955649640323

In [52]:
#we need short:

short_position = 65.0485163778 / 96.6855209534 
short_position

0.6727844638614685

In [53]:
#the delta of forward contract is delta(10 year) - 0.6727 * delta(1 year)

D_forward = D_10 - short_position * D_1
D_forward


-751.53410435245098

In [54]:
# So we short 

D_call / D_forward  

0.97059764075863908

### (f) Hedging with future contract


In [55]:
#caculate the delta of future
Cu = (ZCB_value_tree[2][0]*0.5 + ZCB_value_tree[2][1]*0.5)
Cd = (ZCB_value_tree[2][2]*0.5 + ZCB_value_tree[2][1]*0.5)

D_future = (Cu -Cd)/ (bt[1][0] - bt[1][1])
D_future

-764.52045272169585

In [56]:
# we need short 
D_call / D_future  

0.95411081029597877

### (g) American swaption

In [57]:
# first we need find the swap rate

swap_rate = (1 - disc[-1])/(sum(disc))*2
swap_rate

0.04277240845536031

In [58]:
# test if this is the real par yield or not

BT= bt[:]
BT.reverse()
values = []
value_tree = []
c = 100 * 0.042772408455379121
for branch in BT:
    temp = []
    count = count -1
    if len(values)==0:
        for r in branch:
            values.append( (100+ c/2)/(1+r/2))
    else: 
        for ind in range(len(branch)):

            temp.append( (0.5*values[ind]+0.5*values[ind+1]+c/2)/(1+branch[ind]/2))
        values = temp[:]
    value_tree.append(values)

    
value_tree.reverse()

print(values[0])

99.9999999995


In [59]:
test_tree = bt[:5]

def swap_value(Sub_BT, swap_rate = swap_rate):
    #for a fix recivier 
    sb = Sub_BT[:]
    sb.reverse()
    values = []
    tree_value = []
    for branch in sb:
        temp = []
        if (len(values) == 0):
 
            for r in branch:
                #print(r)
                values.append(100*(swap_rate - r)/2 /(1+r/2))
        else:
            for ind in range(len(branch)):
                temp.append((values[ind]*0.5+0.5* values[ind+1] + 100*(swap_rate - branch[ind])/2 )/(1+branch[ind]/2) )
            values = temp
        #print(values)
        tree_value.append(values)
    return values[0]

def select_sub_tree(time, level, tree = bt):
    #select for a fixed 10 periods
    #all the locations are absolute value
    new_tree=[]
    for i in range(10):
        new_tree.append(bt[time+i][level:(level+1+i)])
        
    return new_tree

In [60]:
sub_tree = bt[:11][:]
sub_tree.reverse()
value = []
current_time = 11
value_tree= []
for branch in sub_tree:
    current_time = current_time - 1
    temp = []
    if len(value) == 0:
        for ind in range(len(branch)):
            value.append(max([swap_value(select_sub_tree(current_time, ind)),0]))
    else:
        for ind in range(len(branch)):
            temp.append(max(swap_value(select_sub_tree(current_time, ind)), (0.5*value[ind]+0.5* value[ind+1])/(1+branch[ind]/2)))
        value = temp
    value_tree.append(value)
print(value[0])
pd.DataFrame(value_tree).T

2.177874775


,0,1,2,3,4,5,6,7,8,9,10
0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.03143438,0.15669464,0.45937328,1.04576970,2.17787477
1,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.06465781,0.28976832,0.78210868,1.67212653,3.38278938,nan
2,0.00000000,0.00000000,0.00000000,0.00000000,0.13271057,0.52821833,1.30599377,2.62119661,4.69834162,nan,nan
3,0.00000000,0.00000000,0.00000000,0.27184415,0.94615754,2.13239441,4.02190439,5.94990815,nan,nan,nan
4,0.00000000,0.00000000,0.55580215,1.65749567,3.39184754,5.35989641,7.14137682,nan,nan,nan,nan
5,0.00000000,1.13437635,2.81887439,4.82010288,6.63500984,8.27589049,nan,nan,nan,nan,nan
6,2.31141293,4.33959284,6.18158533,7.84960010,9.35591376,nan,nan,nan,nan,nan,nan
7,5.78870218,7.47794589,9.00544659,10.38330061,nan,nan,nan,nan,nan,nan,nan
8,8.71034745,10.10381142,11.35936820,nan,nan,nan,nan,nan,nan,nan,nan
9,11.14611282,12.28551184,nan,nan,nan,nan,nan,nan,nan,nan,nan


## Q4 Hull/White

In [61]:
k=0.075
sig = 0.01
M = (np.exp(-k*0.5 ) -1 )
V = (1 - np.exp(-2*k*0.5))*sig**2 / (2*k)
delta_r = np.sqrt(3*V)
j = -0.184/M
j

4.9992416531905617

In [62]:
#thus we will have j = -5 until 5
P = []

j=-5

q_u = 1/6 + (j**2 * M**2 - j*M)/2
q_m = - 1/3 - j**2 * M**2 + 2*j*M
q_d = 7/6 + (j**2 * M**2 - 3*j*M)/2

P.append([q_u,q_m,q_d])

for i in range(9):

    j=i-4
    q_u = 1/6 + (j**2 * M**2 + j*M)/2
    q_m = 2/3 - j**2 * M**2
    q_d = 1/6 + (j**2 * M**2 - j*M)/2
    P.append([q_u,q_m,q_d])
    
j=5

q_u = 7/6 + (j**2 * M**2 + 3*j*M)/2
q_m = - 1/3 - j**2 * M**2 - 2*j*M
q_d = 1/6 + (j**2 * M**2 + j*M)/2

P.append([q_u,q_m,q_d])
P.reverse()
P

[[0.90755793565919729, 0.0008562172857146666, 0.09158584705508814],
 [0.10389270920358513, 0.64499225247611691, 0.25111503832029786],
 [0.11755422223899148, 0.65447480868448238, 0.22797096907652603],
 [0.13257038616130717, 0.66124806311902917, 0.20618155071966354],
 [0.14894120097053223, 0.66531201577975729, 0.18574678324971042],
 [0.16666666666666666, 0.66666666666666663, 0.16666666666666666],
 [0.18574678324971042, 0.66531201577975729, 0.14894120097053223],
 [0.20618155071966354, 0.66124806311902917, 0.13257038616130717],
 [0.22797096907652603, 0.65447480868448238, 0.11755422223899148],
 [0.25111503832029786, 0.64499225247611691, 0.10389270920358513],
 [0.09158584705508814, 0.0008562172857146666, 0.90755793565919729]]

In [63]:
# max knots is 11
f=[100]*13
def discount_HW(HW_BT, Prob = P, final = f):
    bt = HW_BT[:]
    bt.reverse()
    value = []
    tree_value = []
    for branch in bt:
        temp = []
        if len(branch)==11:
            if len(value) == 0:
                for ind in range(len(branch)):
                    value.append(final[ind]*np.exp(-branch[ind]*0.5))
            else:
                for ind in range(len(branch)):
                     
                    if ind == 0:
                        temp.append((value[ind]*Prob[ind][0] + \
                                     value[ind+1]*Prob[ind][1] + value[ind+2]*Prob[ind][2])*np.exp(-branch[ind]*0.5))
                    elif ind == 10:
                        temp.append((value[ind-2]*Prob[ind][0] + \
                                     value[ind-1]*Prob[ind][1] + value[ind]*Prob[ind][2])*np.exp(-branch[ind]*0.5))
                    else:
                        temp.append((value[ind-1]*Prob[ind][0] + \
                                     value[ind]*Prob[ind][1] + value[ind+1]*Prob[ind][2])*np.exp(-branch[ind]*0.5))
                                    
                value = temp
        else:
            if len(value) == 0:
                for ind in range(len(branch)):
                    value.append(final[ind]*np.exp(-branch[ind]*0.5))
            else:
                idx = int((11- len(branch))/2)
                
                for ind in range(len(branch)):
                    temp.append((value[ind]*Prob[idx][0] + \
                                 value[ind+1]*Prob[idx][1]+ value[ind+2]*Prob[idx][2])*np.exp(-branch[ind]*0.5))
                    idx = idx + 1
                value = temp
        tree_value.append(value)
        
    return(value[0])
                    
                                    
def extend_HW(HW_BT , m, delta_r = delta_r):
    new_tree = HW_BT[:]
    if len(new_tree[-1]) == 11:
        new_tree.append([r+m for r in new_tree[-1]])
    else:
        new_branch = [r+delta_r+m for r in new_tree[-1]]
        new_branch.append(new_tree[-1][-1] + m)
        new_branch.append(new_tree[-1][-1] + m - delta_r)
        new_tree.append(new_branch)
    return new_tree
    
    
def find_HW(m, HW_BT, price_ZCB, n,  Prob = P, delta_r = delta_r):
    #only find ZCB
    new_tree = extend_HW(HW_BT, m)
    f=[100]*(2*n-1)
    price = discount_HW(new_tree , Prob, f)
    return(price - price_ZCB)

In [72]:
def find_r0(r):
    return (discount_HW([[r]],P,[100])-disc[0]*100)

r0 = fsolve(find_r0,0.01)[0]

HW_BT = [[r0]]



fsolve(find_HW,0.01,args= (HW_BT, disc[1]*100 , 2))


for i in range(1,20):
    #print(disc[i])
    n = i + 1
    m = fsolve(find_HW, 0.01, args = (HW_BT, disc[i]*100, n))[0]
    HW_BT = extend_HW(HW_BT, m)
    
#HW_BT

In [74]:
pd.DataFrame(HW_BT).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.03315512,0.04629133,0.05946731,0.07267273,0.08589787,0.09913353,0.10034970,0.10155961,0.10275564,0.10393067,0.10507806,0.10619166,0.10726577,0.10829517,0.10927507,0.11020114,0.11106947,0.11187659,0.11261945,0.11329543
1,nan,0.03426998,0.04744595,0.06065138,0.07387651,0.08711217,0.08832834,0.08953825,0.09073428,0.09190931,0.09305670,0.09417030,0.09524441,0.09627381,0.09725371,0.09817978,0.09904811,0.09985523,0.10059810,0.10127408
2,nan,0.02224862,0.03542459,0.04863002,0.06185516,0.07509082,0.07630699,0.07751690,0.07871293,0.07988795,0.08103534,0.08214894,0.08322306,0.08425245,0.08523236,0.08615842,0.08702675,0.08783387,0.08857674,0.08925272
3,nan,nan,0.02340324,0.03660866,0.04983380,0.06306946,0.06428563,0.06549554,0.06669157,0.06786660,0.06901399,0.07012758,0.07120170,0.07223110,0.07321100,0.07413706,0.07500540,0.07581252,0.07655538,0.07723136
4,nan,nan,0.01138188,0.02458731,0.03781244,0.05104810,0.05226427,0.05347418,0.05467021,0.05584524,0.05699263,0.05810623,0.05918034,0.06020974,0.06118964,0.06211571,0.06298404,0.06379116,0.06453403,0.06521001
5,nan,nan,nan,0.01256595,0.02579109,0.03902675,0.04024292,0.04145283,0.04264885,0.04382388,0.04497127,0.04608487,0.04715898,0.04818838,0.04916828,0.05009435,0.05096268,0.05176980,0.05251267,0.05318865
6,nan,nan,nan,0.00054459,0.01376973,0.02700539,0.02822156,0.02943147,0.03062750,0.03180252,0.03294992,0.03406351,0.03513763,0.03616703,0.03714693,0.03807299,0.03894132,0.03974845,0.04049131,0.04116729
7,nan,nan,nan,nan,0.00174837,0.01498403,0.01620020,0.01741011,0.01860614,0.01978117,0.02092856,0.02204216,0.02311627,0.02414567,0.02512557,0.02605164,0.02691997,0.02772709,0.02846995,0.02914593
8,nan,nan,nan,nan,-0.01027299,0.00296267,0.00417884,0.00538875,0.00658478,0.00775981,0.00890720,0.01002080,0.01109491,0.01212431,0.01310421,0.01403028,0.01489861,0.01570573,0.01644860,0.01712458
9,nan,nan,nan,nan,nan,-0.00905868,-0.00784251,-0.00663260,-0.00543657,-0.00426155,-0.00311416,-0.00200056,-0.00092644,0.00010295,0.00108286,0.00200892,0.00287725,0.00368437,0.00442724,0.00510322


## (Simplified) mortgage-backed security (MBS)


In [75]:
r0 = HW_BT[0][0]
def cal_prepay_rate(r, c = r0):
    if r >= c:
        return 0.03
    elif r >= (c-0.005):
        return(0.03 + (c-r) * 4)
    elif r >= (c-0.01):
        return(0.05 + (c-r-0.005)*6)
    elif r >= (c-0.02):
        return(0.08 + (c-r-0.01)*9)
    else:
        return 0.17

In [76]:
HW_BT[0][0]

0.033155119903965045

In [77]:
cal_prepay_rate(0.14,0.05)
principal = 1000000

In [78]:
sch_payment = 1000000 / (sum([(1+0.0575/2)**(-i) for i in range(1,11)]))
sch_payment

def sch_paym(n, principal = principal, rate = 0.0575):
    if n <= 0:
        return 0
    else:
        return(principal/ sum([(1+0.0575/2)**(-i-1) for i in range(n)]))


In [79]:
principal = 1000000
amount_not_due = principal

for i in range(10):
    amount_not_due = amount_not_due*(1+0.0575/2) - sch_payment
    print(amount_not_due)

912266.112011
822009.874742
729158.770652
633638.197319
535371.407503
434279.44748
330281.093606
223292.787058
113228.566697
2.91038304567e-11


In [88]:
#test_rate_path = [0.01, 0.1 ,0.2,0.02,0.03,0.03,0.03,0.03,0.04,0.04]
test_rate_path = [0.06]*10
def value_MBS(rate_path, principal = principal):
    #calculate cashflow
    prepayment_rate = [cal_prepay_rate(i,c=rate_path[0]) for i in rate_path]
    #prepayment_rate = [0] *10
    cashflow = []
    amount_not_due = principal
    n=10
    for pre_rate in prepayment_rate:
        sch_pay = sch_paym(n,amount_not_due)
        n = n - 1
        mortgage_amount = amount_not_due * (1+0.0575/2)
        principal_left = mortgage_amount - sch_pay
        prepay_amount = amount_not_due * pre_rate
        total_cashflow = sch_pay + prepay_amount
        cashflow.append(total_cashflow)
        amount_not_due = principal_left - prepay_amount
        #print(amount_not_due)
    #print(cashflow)
    #Next discount cashflow to time 0
    discount_factor = [np.exp(-r*t*0.5) for r,t in zip(rate_path, range(1,11))]
    MBSvalue = sum([d*c for c,d in zip(cashflow, discount_factor)])
    return(MBSvalue)
    
    
    
value_MBS(test_rate_path)



993898.49316337437

In [89]:
#choice should be a list contain {-1, 0, 1}
test_choice = [-1]*3 + [0]*4 + [1] *3

def pick(choice, prob = P, HW_BT = HW_BT):
    pointer = 0
    rate_path = [HW_BT[0][0]]
    time = 1
    for c in choice:
        if pointer == 5:
            pointer = pointer + (c-1)
        elif pointer == -5:
            pointer = pointer + (c+1)
        else:
            pointer = pointer +c
        

            absolute_loca = min(time ,5) - pointer

        #print(absolute_loca)
        rate_path.append(HW_BT[time][absolute_loca])
        time = time + 1 
        
    p=1
    pointer= 0
    for c in choice:
        p = p * prob[ 5 - pointer][1 - c]
        
        if pointer == 5:
            pointer = pointer + (c-1)
        elif pointer == -5:
            pointer = pointer + (c+1)
        else:
            pointer = pointer +c
        
        
    
    return rate_path,p



def extend(orig):
    new = []
    for i in orig:
        new.append(i*10 + 1)
        new.append(i*10 + 2)
        new.append(i*10 + 3)
    return new

def converter(n):
    return [int(i)-2 for i in list(str(n))]


    
    

In [90]:
l=extend([1,2,3])
for i in range(8):
    l = extend(l)
    


choice_of_path = [converter(i) for i in l]
choice_of_path[0]

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

In [91]:
Exact_value = 0
p = 0
for path in choice_of_path:
    
    rat_p, pro = pick(path)
    #print(path)
    Exact_value = Exact_value + value_MBS(rat_p) * pro
    p = p + pro
print("The exact price of the MBS:", Exact_value)
print("Totoal probability of all the paths:",p)

The exact price of the MBS: 1043402.62547
Totoal probability of all the paths: 1.0


### (b) Monte Carlo Simulation

we need 1000 * 10 random varables.

In [92]:
np.random.seed(0)
ran_num = np.random.rand(10000)

In [93]:
def choice_path(ran, prob = P):
    pointer = 0
    choice = []
    for x in ran:
        if x <= prob[5-pointer][0]:
            choice.append(1)
        elif (x<= (prob[5-pointer][1]+prob[5-pointer][0])) and (x > prob[5-pointer][0]):
            choice.append(0)
        else:
            choice.append(-1)
    return(choice)

In [94]:
choice_path(ran_num[:10])

[0, 0, 0, 0, 0, 0, 0, -1, -1, 0]

In [95]:
paths = []
for i in range(1000):
    paths.append(choice_path(ran_num[(10*i) : (10*i +10)]))

In [96]:
mbs_prices= [value_MBS(pick(path)[0]) for path in paths]
print(np.mean(mbs_prices))
print(np.std(mbs_prices))


1040986.62519
32412.8909819


### (c)

In [97]:
anti_ran = [1- n for n in ran_num[:5000]]
paths = []
for i in range(500):
    
    paths.append(choice_path(ran_num[(10*i) : (10*i +10)]))
    
for i in range(500):
    
    paths.append(choice_path([1-i for i in ran_num[(10*i) : (10*i +10)]]))

In [98]:
value_MBS([-1, 0, -1, 0, 0, 0, 1, -1, -1, -1])

27390855.653459404

In [99]:
mbs_prices= [value_MBS(pick(path)[0]) for path in paths]
print(np.mean(mbs_prices))
print(np.std(mbs_prices))


1043165.28013
31912.0097872
